In [1]:
# import breast caner dataset from sklearn

from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import statsmodels.api as sm
import nbformat as nbf
import sympy as sp
from sympy import symbols, Matrix, diff
import random
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.decomposition import PCA
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV


In [2]:
# Load the breast cancer dataset in a good format

data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

---

## <span style="color:darkblue"> Task - 1 : Formulation of this as Logistic Regression with L2 Penalty </span>

---


#### Features are considered as X = [x_1, x_2, ......, x_n]
#### Weights for those features w = [w_1, w_2, ......, w_n]
#### bias is considered as = b


So, the Linear Score would be : 

![image44](Equations/44.png)

##### If the **S_i = Positive -----> Malignant**, **S_i = Negative -----> Benin**

But this is the classification problem, so we need to convert the linear model scores(regression) as classification, so we are introducing the new function called as **Logistic Function**

### Logistic Function : 

It tells about the probability of location of the given data point, and it doesn't give the exact location, it just tells the probability.

So, the scores are defined as follows : 

![image45](Equations/45.png)

#### Why Logistic ? 

1. Smooth and Differentiable ----> Easy to calculate the gradient
2. Gives the probability of dataset not just +1 or -1 


### Loss Function :

![image46](Equations/46.png)


For all the datapoints : 

![image47](Equations/47.png)


Now we are adding the L2 Penalty, because it has many features around(20), some weights might become very big, so it may lead to overfitting.

After adding L2 : 

1. Small weights discipline
2. Big weights got punished 
3. Smooth gradient and good fitting of the model 


Our **Final Objective Function** :

![image48](Equations/48.png)

### How this is Convex Model  ?


![image49](Equations/49.png)

As, second derivative is > 0, then it is **Convex Function**

And we know that, L2 is also a **Convex Function**

![image50](Equations/50.png)

**So, Finally this Logistic Model is Convex**

---

# <span style="color:darkblue">Task 2: Gradient Derivation for Logistic Loss Function</span>

---

![image51](Equations/51.png)

**Gradient Wrt W :**

![image52](Equations/52.png)

**Gradient wrt b :**

![image53](Equations/53.png)


### Update Rules :

**Full Batch Gradient Descent :**

![image54](Equations/54.png)

**Stochastic Gradient Descent :**

![image55](Equations/55.png)

**Mini Batch Gradient Descent :**

![image56](Equations/56.png)

**SAGA :**

![image57](Equations/57.png)





### Overall Updates Would be : 


![image58](Equations/58.png)

---

## <span style="color:darkblue"> Task - 3 : Implementation of All Gradients 

---

In [3]:
data = load_breast_cancer()
X = data.data
y = 2*data.target - 1  # convert {0,1} -> {-1,+1}

X_mean = X.mean(axis=0)
X_std = X.std(axis=0) + 1e-8
X = (X - X_mean)/X_std

n, d = X.shape

def sigmoid(s):
    return 1 / (1 + np.exp(-s))


lam = 0.1
def logistic_loss_grad(w, b, X, y, lam):
    n = X.shape[0]
    s = X.dot(w) + b
    probs = sigmoid(-y*s)
    grad_w = (-1/n)*(X.T @ (y*probs)) + lam*w
    grad_b = (-1/n)*np.sum(y*probs)
    loss = (1/n)*np.sum(np.log(1 + np.exp(-y*s))) + (lam/2)*np.sum(w**2)
    return grad_w, grad_b, loss



#### <span style="color:red"> Implementation of Full Batch

In [4]:
eta = 0.01
n_iter = 10000 # number of iterations

w = np.zeros(d)
b = 0.0
loss_history_gd = []

for t in range(n_iter):
    grad_w, grad_b, loss = logistic_loss_grad(w, b, X, y, lam)
    w = w - eta * grad_w
    b = b - eta * grad_b
    loss_history_gd.append(loss)
    if t % 10 == 0:
        print(f"Iteration {t}, Loss: {loss:.4f}")


Iteration 0, Loss: 0.6931
Iteration 10, Loss: 0.5429
Iteration 20, Loss: 0.4587
Iteration 30, Loss: 0.4055
Iteration 40, Loss: 0.3689
Iteration 50, Loss: 0.3422
Iteration 60, Loss: 0.3219
Iteration 70, Loss: 0.3059
Iteration 80, Loss: 0.2930
Iteration 90, Loss: 0.2823
Iteration 100, Loss: 0.2734
Iteration 110, Loss: 0.2659
Iteration 120, Loss: 0.2595
Iteration 130, Loss: 0.2539
Iteration 140, Loss: 0.2490
Iteration 150, Loss: 0.2447
Iteration 160, Loss: 0.2409
Iteration 170, Loss: 0.2376
Iteration 180, Loss: 0.2346
Iteration 190, Loss: 0.2319
Iteration 200, Loss: 0.2294
Iteration 210, Loss: 0.2272
Iteration 220, Loss: 0.2253
Iteration 230, Loss: 0.2234
Iteration 240, Loss: 0.2218
Iteration 250, Loss: 0.2203
Iteration 260, Loss: 0.2189
Iteration 270, Loss: 0.2176
Iteration 280, Loss: 0.2165
Iteration 290, Loss: 0.2154
Iteration 300, Loss: 0.2144
Iteration 310, Loss: 0.2135
Iteration 320, Loss: 0.2126
Iteration 330, Loss: 0.2119
Iteration 340, Loss: 0.2111
Iteration 350, Loss: 0.2104
Ite


#### <span style="color:red"> Implementation of SGD

In [5]:
eta = 0.01
n_iter = 100000  # SGD usually needs more iterations

w_sgd = np.zeros(d)
b_sgd = 0.0
loss_history_sgd = []

for t in range(n_iter):
    i_t = np.random.randint(0, n)
    x_i, y_i = X[i_t], y[i_t]
    s_i = np.dot(w_sgd, x_i) + b_sgd
    grad_w = -y_i * sigmoid(-y_i*s_i) * x_i + lam*w_sgd
    grad_b = -y_i * sigmoid(-y_i*s_i)
    
    w_sgd = w_sgd - eta*grad_w
    b_sgd = b_sgd - eta*grad_b
    
    # Compute full loss for tracking convergence
    _, _, loss = logistic_loss_grad(w_sgd, b_sgd, X, y, lam)
    loss_history_sgd.append(loss)
    if t % 100 == 0:
        print(f"Iteration {t}, Loss: {loss:.4f}")


Iteration 0, Loss: 0.6703
Iteration 100, Loss: 0.2791
Iteration 200, Loss: 0.2289
Iteration 300, Loss: 0.2122
Iteration 400, Loss: 0.2068
Iteration 500, Loss: 0.2041
Iteration 600, Loss: 0.2019
Iteration 700, Loss: 0.2003
Iteration 800, Loss: 0.1998
Iteration 900, Loss: 0.1999
Iteration 1000, Loss: 0.1997
Iteration 1100, Loss: 0.2012
Iteration 1200, Loss: 0.1995
Iteration 1300, Loss: 0.2001
Iteration 1400, Loss: 0.2004
Iteration 1500, Loss: 0.1993
Iteration 1600, Loss: 0.1985
Iteration 1700, Loss: 0.1987
Iteration 1800, Loss: 0.1994
Iteration 1900, Loss: 0.1998
Iteration 2000, Loss: 0.1991
Iteration 2100, Loss: 0.1987
Iteration 2200, Loss: 0.1988
Iteration 2300, Loss: 0.1985
Iteration 2400, Loss: 0.1979
Iteration 2500, Loss: 0.1974
Iteration 2600, Loss: 0.1974
Iteration 2700, Loss: 0.1983
Iteration 2800, Loss: 0.1994
Iteration 2900, Loss: 0.1986
Iteration 3000, Loss: 0.1996
Iteration 3100, Loss: 0.1984
Iteration 3200, Loss: 0.1976
Iteration 3300, Loss: 0.1979
Iteration 3400, Loss: 0.19


#### <span style="color:red"> Implementation of Mini Batch 32

In [6]:
eta = 0.01
batch_size = 32
n_iter = 10000

w_batch = np.zeros(d)
b_batch = 0.0
loss_history_batch = []

for t in range(n_iter):
    batch_idx = np.random.choice(n, batch_size, replace=False)
    X_b, y_b = X[batch_idx], y[batch_idx]
    
    s_b = X_b.dot(w_batch) + b_batch
    probs_b = sigmoid(-y_b*s_b)
    grad_w = (-1/batch_size) * (X_b.T @ (y_b*probs_b)) + lam*w_batch
    grad_b = (-1/batch_size) * np.sum(y_b*probs_b)
    
    w_batch = w_batch - eta*grad_w
    b_batch = b_batch - eta*grad_b
    
    # Track full dataset loss
    _, _, loss = logistic_loss_grad(w_batch, b_batch, X, y, lam)
    loss_history_batch.append(loss)
    if t % 20 == 0:
        print(f"Iteration {t}, Loss: {loss:.4f}")


Iteration 0, Loss: 0.6745
Iteration 20, Loss: 0.4523
Iteration 40, Loss: 0.3675
Iteration 60, Loss: 0.3204
Iteration 80, Loss: 0.2921
Iteration 100, Loss: 0.2729
Iteration 120, Loss: 0.2586
Iteration 140, Loss: 0.2481
Iteration 160, Loss: 0.2400
Iteration 180, Loss: 0.2340
Iteration 200, Loss: 0.2286
Iteration 220, Loss: 0.2248
Iteration 240, Loss: 0.2218
Iteration 260, Loss: 0.2190
Iteration 280, Loss: 0.2163
Iteration 300, Loss: 0.2143
Iteration 320, Loss: 0.2127
Iteration 340, Loss: 0.2112
Iteration 360, Loss: 0.2098
Iteration 380, Loss: 0.2085
Iteration 400, Loss: 0.2073
Iteration 420, Loss: 0.2065
Iteration 440, Loss: 0.2057
Iteration 460, Loss: 0.2051
Iteration 480, Loss: 0.2045
Iteration 500, Loss: 0.2039
Iteration 520, Loss: 0.2035
Iteration 540, Loss: 0.2029
Iteration 560, Loss: 0.2026
Iteration 580, Loss: 0.2022
Iteration 600, Loss: 0.2020
Iteration 620, Loss: 0.2018
Iteration 640, Loss: 0.2015
Iteration 660, Loss: 0.2011
Iteration 680, Loss: 0.2009
Iteration 700, Loss: 0.200


#### <span style="color:red"> Implementation of Mini Batch 64

In [7]:
eta = 0.01
batch_size = 64
n_iter = 10000

w_batch = np.zeros(d)
b_batch = 0.0
loss_history_batch = []

for t in range(n_iter):
    batch_idx = np.random.choice(n, batch_size, replace=False)
    X_b, y_b = X[batch_idx], y[batch_idx]
    
    s_b = X_b.dot(w_batch) + b_batch
    probs_b = sigmoid(-y_b*s_b)
    grad_w = (-1/batch_size) * (X_b.T @ (y_b*probs_b)) + lam*w_batch
    grad_b = (-1/batch_size) * np.sum(y_b*probs_b)
    
    w_batch = w_batch - eta*grad_w
    b_batch = b_batch - eta*grad_b
    
    # Track full dataset loss
    _, _, loss = logistic_loss_grad(w_batch, b_batch, X, y, lam)
    loss_history_batch.append(loss)
    if t % 20 == 0:
        print(f"Iteration {t}, Loss: {loss:.4f}")


Iteration 0, Loss: 0.6746
Iteration 20, Loss: 0.4511
Iteration 40, Loss: 0.3652
Iteration 60, Loss: 0.3197
Iteration 80, Loss: 0.2915
Iteration 100, Loss: 0.2730
Iteration 120, Loss: 0.2593
Iteration 140, Loss: 0.2489
Iteration 160, Loss: 0.2410
Iteration 180, Loss: 0.2342
Iteration 200, Loss: 0.2292
Iteration 220, Loss: 0.2250
Iteration 240, Loss: 0.2215
Iteration 260, Loss: 0.2187
Iteration 280, Loss: 0.2163
Iteration 300, Loss: 0.2142
Iteration 320, Loss: 0.2125
Iteration 340, Loss: 0.2110
Iteration 360, Loss: 0.2098
Iteration 380, Loss: 0.2087
Iteration 400, Loss: 0.2077
Iteration 420, Loss: 0.2067
Iteration 440, Loss: 0.2060
Iteration 460, Loss: 0.2052
Iteration 480, Loss: 0.2046
Iteration 500, Loss: 0.2041
Iteration 520, Loss: 0.2036
Iteration 540, Loss: 0.2032
Iteration 560, Loss: 0.2028
Iteration 580, Loss: 0.2024
Iteration 600, Loss: 0.2021
Iteration 620, Loss: 0.2018
Iteration 640, Loss: 0.2016
Iteration 660, Loss: 0.2013
Iteration 680, Loss: 0.2011
Iteration 700, Loss: 0.200


#### <span style="color:red"> Implementation of SAGA

In [8]:
eta = 0.01         
n_iter = 10000      
w = np.zeros(d)    
b = 0.0         

g_old = np.zeros((n, d))
loss_history_saga = []

# Step: SAGA iterations (with multiple iterations)
for t in range(n_iter):
    i_t = np.random.randint(0, n)
    x_i, y_i = X[i_t], y[i_t]

    s_i = np.dot(w, x_i) + b
    g_i_t = -y_i * sigmoid(-y_i*s_i) * x_i + lam*w
    g_bar_old = np.mean(g_old, axis=0)
    w = w - eta * (g_i_t - g_old[i_t] + g_bar_old)
    s_all = X.dot(w) + b
    b_grad = -y_i*sigmoid(-y_i*s_i) + np.mean(-y*sigmoid(-y*s_all))
    b = b - eta * b_grad
    
    g_old[i_t] = g_i_t
    _, _, loss = logistic_loss_grad(w, b, X, y, lam)
    loss_history_saga.append(loss)
    if t % 100 == 0:
        print(f"Iteration {t}, Loss: {loss:.4f}")




Iteration 0, Loss: 0.6706
Iteration 100, Loss: 0.2631
Iteration 200, Loss: 0.2256
Iteration 300, Loss: 0.2142
Iteration 400, Loss: 0.2075
Iteration 500, Loss: 0.2031
Iteration 600, Loss: 0.1995
Iteration 700, Loss: 0.1984
Iteration 800, Loss: 0.1985
Iteration 900, Loss: 0.1977
Iteration 1000, Loss: 0.1977
Iteration 1100, Loss: 0.1978
Iteration 1200, Loss: 0.1973
Iteration 1300, Loss: 0.1970
Iteration 1400, Loss: 0.1970
Iteration 1500, Loss: 0.1969
Iteration 1600, Loss: 0.1969
Iteration 1700, Loss: 0.1969
Iteration 1800, Loss: 0.1969
Iteration 1900, Loss: 0.1968
Iteration 2000, Loss: 0.1969
Iteration 2100, Loss: 0.1968
Iteration 2200, Loss: 0.1969
Iteration 2300, Loss: 0.1970
Iteration 2400, Loss: 0.1968
Iteration 2500, Loss: 0.1969
Iteration 2600, Loss: 0.1968
Iteration 2700, Loss: 0.1968
Iteration 2800, Loss: 0.1968
Iteration 2900, Loss: 0.1968
Iteration 3000, Loss: 0.1968
Iteration 3100, Loss: 0.1968
Iteration 3200, Loss: 0.1968
Iteration 3300, Loss: 0.1968
Iteration 3400, Loss: 0.19

---

## <span style="color:darkblue"> Task - 4 : Comparison of Convergence Rates, F1, Accuracy, Precision ....

---

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
n_train = X_train.shape[0]

In [10]:
# GD
start = time.time()
w_gd = np.zeros(d)
b_gd = 0.0
loss_history_gd = []

for t in range(n_iter):
    grad_w, grad_b, loss = logistic_loss_grad(w_gd, b_gd, X_train, y_train, lam)
    w_gd -= eta*grad_w
    b_gd -= eta*grad_b
    loss_history_gd.append(loss)
gd_time = time.time() - start
print("GD finished in", gd_time, "seconds")


GD finished in 0.38208675384521484 seconds


In [11]:
# SGD
start = time.time()
w_sgd = np.zeros(d)
b_sgd = 0.0
loss_history_sgd = []

for t in range(n_iter):
    i_t = np.random.randint(0, n_train)
    x_i, y_i = X_train[i_t], y_train[i_t]
    s_i = np.dot(w_sgd, x_i) + b_sgd
    grad_w = -y_i * sigmoid(-y_i*s_i) * x_i + lam*w_sgd
    grad_b = -y_i * sigmoid(-y_i*s_i)
    w_sgd -= eta*grad_w
    b_sgd -= eta*grad_b
    _, _, loss = logistic_loss_grad(w_sgd, b_sgd, X_train, y_train, lam)
    loss_history_sgd.append(loss)
sgd_time = time.time() - start
print("SGD finished in", sgd_time, "seconds")


SGD finished in 0.5126848220825195 seconds


In [12]:
# Mini-batch GD (32)
start = time.time()
batch_size = 32
w_batch32 = np.zeros(d)
b_batch32 = 0.0
loss_history_batch32 = []

for t in range(n_iter):
    idx = np.random.choice(n_train, batch_size, replace=False)
    X_b, y_b = X_train[idx], y_train[idx]
    s_b = X_b.dot(w_batch32) + b_batch32
    probs = sigmoid(-y_b*s_b)
    grad_w = (-1/batch_size)*(X_b.T @ (y_b*probs)) + lam*w_batch32
    grad_b = (-1/batch_size)*np.sum(y_b*probs)
    w_batch32 -= eta*grad_w
    b_batch32 -= eta*grad_b
    _, _, loss = logistic_loss_grad(w_batch32, b_batch32, X_train, y_train, lam)
    loss_history_batch32.append(loss)
mb32_time = time.time() - start
print("Mini-batch 32 finished in", mb32_time, "seconds")


Mini-batch 32 finished in 0.8195517063140869 seconds


In [13]:
# Mini-batch GD (64)
start = time.time()
batch_size = 64
w_batch64 = np.zeros(d)
b_batch64 = 0.0
loss_history_batch64 = []

for t in range(n_iter):
    idx = np.random.choice(n_train, batch_size, replace=False)
    X_b, y_b = X_train[idx], y_train[idx]
    s_b = X_b.dot(w_batch64) + b_batch64
    probs = sigmoid(-y_b*s_b)
    grad_w = (-1/batch_size)*(X_b.T @ (y_b*probs)) + lam*w_batch64
    grad_b = (-1/batch_size)*np.sum(y_b*probs)
    w_batch64 -= eta*grad_w
    b_batch64 -= eta*grad_b
    _, _, loss = logistic_loss_grad(w_batch64, b_batch64, X_train, y_train, lam)
    loss_history_batch64.append(loss)
mb64_time = time.time() - start
print("Mini-batch 64 finished in", mb64_time, "seconds")


Mini-batch 64 finished in 0.8152902126312256 seconds


In [14]:
# SAGA
start = time.time()
w_saga = np.zeros(d)
b_saga = 0.0
g_old = np.zeros((n_train,d))
loss_history_saga = []

for t in range(n_iter):
    i_t = np.random.randint(0, n_train)
    x_i, y_i = X_train[i_t], y_train[i_t]
    s_i = np.dot(w_saga, x_i) + b_saga
    g_i_t = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_saga
    g_bar = np.mean(g_old, axis=0)
    w_saga -= eta*(g_i_t - g_old[i_t] + g_bar)
    s_all = X_train.dot(w_saga) + b_saga
    b_grad = -y_i*sigmoid(-y_i*s_i) + np.mean(-y_train*sigmoid(-y_train*s_all))
    b_saga -= eta*b_grad
    g_old[i_t] = g_i_t
    _, _, loss = logistic_loss_grad(w_saga, b_saga, X_train, y_train, lam)
    loss_history_saga.append(loss)
saga_time = time.time() - start
print("SAGA finished in", saga_time, "seconds")


SAGA finished in 0.9647159576416016 seconds


In [15]:
import plotly.graph_objects as go
import numpy as np

methods = {
    "GD": (loss_history_gd, gd_time),
    "SGD": (loss_history_sgd, sgd_time),
    "Mini-batch 32": (loss_history_batch32, mb32_time),
    "Mini-batch 64": (loss_history_batch64, mb64_time),
    "SAGA": (loss_history_saga, saga_time)
}

# ----- Loss vs Iterations -----
for method, (loss_hist, _) in methods.items():
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=loss_hist, mode='lines', name=method))
    fig.update_layout(
        title=f'{method} - Loss vs Iterations',
        xaxis_title='Iterations',
        yaxis_title='Loss',
        width=700, height=450
    )
    fig.show()

# ----- Loss vs Time -----
for method, (loss_hist, t) in methods.items():
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.linspace(0, t, len(loss_hist)), y=loss_hist, mode='lines', name=method))
    fig.update_layout(
        title=f'{method} - Loss vs Time',
        xaxis_title='Time (s)',
        yaxis_title='Loss',
        width=700, height=450
    )
    fig.show()


In [16]:

def predict(X, w, b):
    return np.where(sigmoid(X.dot(w)+b) >= 0.5, 1, -1)

algorithms = {
    "GD": (w_gd, b_gd),
    "SGD": (w_sgd, b_sgd),
    "Mini-batch 32": (w_batch32, b_batch32),
    "Mini-batch 64": (w_batch64, b_batch64),
    "SAGA": (w_saga, b_saga)
}

# Collect results
results = []
for name, (w_alg, b_alg) in algorithms.items():
    y_pred = predict(X_test, w_alg, b_alg)
    results.append({
        "Algorithm": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred)
    })

# Convert to DataFrame and display as table
df_results = pd.DataFrame(results)
df_results = df_results.set_index("Algorithm")
df_results.style.format("{:.4f}")


,Accuracy,Precision,Recall,F1-score
Algorithm,,,,
GD,0.9825,0.9726,1.0000,0.9861
SGD,0.9825,0.9726,1.0000,0.9861
Mini-batch 32,0.9737,0.9595,1.0000,0.9793
Mini-batch 64,0.9825,0.9726,1.0000,0.9861
SAGA,0.9825,0.9726,1.0000,0.9861


### But wait, How can all the models without any feature engineering would get this accuracy, let's check overfitting

In [17]:
# ----- Train & Test Loss Tracking -----
loss_train = {
    "GD": [],
    "SGD": [],
    "Mini-batch 32": [],
    "Mini-batch 64": [],
    "SAGA": []
}

loss_test = {
    "GD": [],
    "SGD": [],
    "Mini-batch 32": [],
    "Mini-batch 64": [],
    "SAGA": []
}


In [18]:
w_gd = np.zeros(d)
b_gd = 0.0
eta = 0.01

for t in range(n_iter):
    grad_w, grad_b, loss = logistic_loss_grad(w_gd, b_gd, X_train, y_train, lam)
    w_gd -= eta*grad_w
    b_gd -= eta*grad_b

    # Track losses
    _, _, train_loss = logistic_loss_grad(w_gd, b_gd, X_train, y_train, lam)
    _, _, test_loss  = logistic_loss_grad(w_gd, b_gd, X_test, y_test, lam)
    loss_train["GD"].append(train_loss)
    loss_test["GD"].append(test_loss)


In [19]:
w_sgd = np.zeros(d)
b_sgd = 0.0
eta = 0.01

for t in range(n_iter):
    i_t = np.random.randint(0, n_train)
    x_i, y_i = X_train[i_t], y_train[i_t]
    s_i = np.dot(w_sgd, x_i) + b_sgd
    grad_w = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_sgd
    grad_b = -y_i*sigmoid(-y_i*s_i)
    w_sgd -= eta*grad_w
    b_sgd -= eta*grad_b

    # Track losses
    _, _, train_loss = logistic_loss_grad(w_sgd, b_sgd, X_train, y_train, lam)
    _, _, test_loss  = logistic_loss_grad(w_sgd, b_sgd, X_test, y_test, lam)
    loss_train["SGD"].append(train_loss)
    loss_test["SGD"].append(test_loss)


In [20]:
w_mb32 = np.zeros(d)
b_mb32 = 0.0
batch_size = 32
eta = 0.01

for t in range(n_iter):
    idx = np.random.choice(n_train, batch_size, replace=False)
    X_b, y_b = X_train[idx], y_train[idx]
    s_b = X_b.dot(w_mb32) + b_mb32
    probs = sigmoid(-y_b*s_b)
    grad_w = (-1/batch_size)*(X_b.T @ (y_b*probs)) + lam*w_mb32
    grad_b = (-1/batch_size)*np.sum(y_b*probs)
    w_mb32 -= eta*grad_w
    b_mb32 -= eta*grad_b

    _, _, train_loss = logistic_loss_grad(w_mb32, b_mb32, X_train, y_train, lam)
    _, _, test_loss  = logistic_loss_grad(w_mb32, b_mb32, X_test, y_test, lam)
    loss_train["Mini-batch 32"].append(train_loss)
    loss_test["Mini-batch 32"].append(test_loss)


In [21]:
w_mb64 = np.zeros(d)
b_mb64 = 0.0
batch_size = 64
eta = 0.01

for t in range(n_iter):
    idx = np.random.choice(n_train, batch_size, replace=False)
    X_b, y_b = X_train[idx], y_train[idx]
    s_b = X_b.dot(w_mb64) + b_mb64
    probs = sigmoid(-y_b*s_b)
    grad_w = (-1/batch_size)*(X_b.T @ (y_b*probs)) + lam*w_mb64
    grad_b = (-1/batch_size)*np.sum(y_b*probs)
    w_mb64 -= eta*grad_w
    b_mb64 -= eta*grad_b

    _, _, train_loss = logistic_loss_grad(w_mb64, b_mb64, X_train, y_train, lam)
    _, _, test_loss  = logistic_loss_grad(w_mb64, b_mb64, X_test, y_test, lam)
    loss_train["Mini-batch 64"].append(train_loss)
    loss_test["Mini-batch 64"].append(test_loss)


In [22]:
w_saga = np.zeros(d)
b_saga = 0.0
g_old = np.zeros((n_train, d))
eta = 0.01

for t in range(n_iter):
    i_t = np.random.randint(0, n_train)
    x_i, y_i = X_train[i_t], y_train[i_t]
    s_i = np.dot(w_saga, x_i) + b_saga
    g_i_t = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_saga
    g_bar = np.mean(g_old, axis=0)
    w_saga -= eta*(g_i_t - g_old[i_t] + g_bar)
    s_all = X_train.dot(w_saga) + b_saga
    b_grad = -y_i*sigmoid(-y_i*s_i) + np.mean(-y_train*sigmoid(-y_train*s_all))
    b_saga -= eta*b_grad
    g_old[i_t] = g_i_t

    _, _, train_loss = logistic_loss_grad(w_saga, b_saga, X_train, y_train, lam)
    _, _, test_loss  = logistic_loss_grad(w_saga, b_saga, X_test, y_test, lam)
    loss_train["SAGA"].append(train_loss)
    loss_test["SAGA"].append(test_loss)
    


In [23]:
# ----- Train & Test Loss  values in single table -----

import pandas as pd

loss_summary = []
for method in loss_train.keys():
    loss_summary.append({
        "Method": method,
        "Final Train Loss": loss_train[method][-1],
        "Final Test Loss": loss_test[method][-1]
    })
df_loss_summary = pd.DataFrame(loss_summary)
df_loss_summary = df_loss_summary.set_index("Method")
df_loss_summary.style.format("{:.4f}")


,Final Train Loss,Final Test Loss
Method,,
GD,0.1999,0.1858
SGD,0.2008,0.1881
Mini-batch 32,0.2000,0.1856
Mini-batch 64,0.1999,0.1858
SAGA,0.1999,0.1858


### No much difference in train loss and Test loss .... So, in this perspective **no overfitting** .... Something Wrong .... Let's go with **K-Fold Cross Validation**

**K-Fold Cross Valiation :**

In [24]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Store metrics for all folds
results_cv = {
    "GD": [],
    "SGD": [],
    "Mini-batch 32": [],
    "Mini-batch 64": [],
    "SAGA": []
}


In [25]:
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    n_train = X_train.shape[0]
    
    # ----- Initialize all models -----
    w_gd = np.zeros(d); b_gd = 0.0
    w_sgd = np.zeros(d); b_sgd = 0.0
    w_mb32 = np.zeros(d); b_mb32 = 0.0
    w_mb64 = np.zeros(d); b_mb64 = 0.0
    w_saga = np.zeros(d); b_saga = 0.0; g_old = np.zeros((n_train,d))

    # ----- Hyperparameters -----
    eta = 0.01
    n_iter_cv = 10000
    batch32 = 32
    batch64 = 64


In [26]:
# ---- GD ----
for t in range(n_iter_cv):
    grad_w, grad_b, _ = logistic_loss_grad(w_gd, b_gd, X_train, y_train, lam)
    w_gd -= eta*grad_w
    b_gd -= eta*grad_b

# ----- SGD ----
for t in range(n_iter_cv):
    i_t = np.random.randint(0, n_train)
    x_i, y_i = X_train[i_t], y_train[i_t]
    s_i = np.dot(w_sgd, x_i) + b_sgd
    grad_w = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_sgd
    grad_b = -y_i*sigmoid(-y_i*s_i)
    w_sgd -= eta*grad_w
    b_sgd -= eta*grad_b

# ----- Mini-batch 32 ----
for t in range(n_iter_cv):
    idx = np.random.choice(n_train, batch32, replace=False)
    X_b, y_b = X_train[idx], y_train[idx]
    s_b = X_b.dot(w_mb32) + b_mb32
    probs = sigmoid(-y_b*s_b)
    grad_w = (-1/batch32)*(X_b.T @ (y_b*probs)) + lam*w_mb32
    grad_b = (-1/batch32)*np.sum(y_b*probs)
    w_mb32 -= eta*grad_w
    b_mb32 -= eta*grad_b

# ----- Mini-batch 64 ----
for t in range(n_iter_cv):
    idx = np.random.choice(n_train, batch64, replace=False)
    X_b, y_b = X_train[idx], y_train[idx]
    s_b = X_b.dot(w_mb64) + b_mb64
    probs = sigmoid(-y_b*s_b)
    grad_w = (-1/batch64)*(X_b.T @ (y_b*probs)) + lam*w_mb64
    grad_b = (-1/batch64)*np.sum(y_b*probs)
    w_mb64 -= eta*grad_w
    b_mb64 -= eta*grad_b

# ----- SAGA ----
for t in range(n_iter_cv):
    i_t = np.random.randint(0, n_train)
    x_i, y_i = X_train[i_t], y_train[i_t]
    s_i = np.dot(w_saga, x_i) + b_saga
    g_i_t = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_saga
    g_bar = np.mean(g_old, axis=0)
    w_saga -= eta*(g_i_t - g_old[i_t] + g_bar)
    s_all = X_train.dot(w_saga) + b_saga
    b_grad = -y_i*sigmoid(-y_i*s_i) + np.mean(-y_train*sigmoid(-y_train*s_all))
    b_saga -= eta*b_grad
    g_old[i_t] = g_i_t


In [27]:
    algorithms = {
        "GD": (w_gd, b_gd),
        "SGD": (w_sgd, b_sgd),
        "Mini-batch 32": (w_mb32, b_mb32),
        "Mini-batch 64": (w_mb64, b_mb64),
        "SAGA": (w_saga, b_saga)
    }

    for name, (w_alg, b_alg) in algorithms.items():
        y_pred = predict(X_test, w_alg, b_alg)
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        results_cv[name].append([acc, prec, rec, f1])


In [28]:
# Compute mean metrics across folds
summary = []
for name in results_cv:
    arr = np.array(results_cv[name])
    summary.append({
        "Algorithm": name,
        "Accuracy": arr[:,0].mean(),
        "Precision": arr[:,1].mean(),
        "Recall": arr[:,2].mean(),
        "F1-score": arr[:,3].mean()
    })

df_cv = pd.DataFrame(summary).set_index("Algorithm")
df_cv.style.format("{:.4f}")


,Accuracy,Precision,Recall,F1-score
Algorithm,,,,
GD,0.9381,0.9054,1.0000,0.9504
SGD,0.9381,0.9054,1.0000,0.9504
Mini-batch 32,0.9381,0.9054,1.0000,0.9504
Mini-batch 64,0.9381,0.9054,1.0000,0.9504
SAGA,0.9381,0.9054,1.0000,0.9504


## Now, it's seems fine as the model gives the almost similiar values 

---

## <span style="color:darkblue"> Task - 5 : Effect of Regularization Parameter over {0.001,0.01,0.1,1}

---

In [29]:
lambdas = [0.001, 0.01, 0.1, 1.0]
results_lambda = {}


from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for lam in lambdas:
    results_lambda[lam] = { "GD": [], "SGD": [], "Mini-batch 32": [], 
                             "Mini-batch 64": [], "SAGA": [] }
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        n_train = X_train.shape[0]

        # ----- Initialize all algorithms -----
        w_gd, b_gd = np.zeros(d), 0.0
        w_sgd, b_sgd = np.zeros(d), 0.0
        w_mb32, b_mb32 = np.zeros(d), 0.0
        w_mb64, b_mb64 = np.zeros(d), 0.0
        w_saga, b_saga = np.zeros(d), 0.0
        g_old = np.zeros((n_train,d))

        # Hyperparameters
        eta = 0.01
        n_iter_cv = 1000
        batch32, batch64 = 32, 64

        # ----- Train GD -----
        for t in range(n_iter_cv):
            grad_w, grad_b, _ = logistic_loss_grad(w_gd, b_gd, X_train, y_train, lam)
            w_gd -= eta*grad_w
            b_gd -= eta*grad_b

        # ----- Train SGD -----
        for t in range(n_iter_cv):
            i_t = np.random.randint(0, n_train)
            x_i, y_i = X_train[i_t], y_train[i_t]
            s_i = np.dot(w_sgd, x_i) + b_sgd
            grad_w = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_sgd
            grad_b = -y_i*sigmoid(-y_i*s_i)
            w_sgd -= eta*grad_w
            b_sgd -= eta*grad_b

        # ----- Mini-batch 32 -----
        for t in range(n_iter_cv):
            idx = np.random.choice(n_train, batch32, replace=False)
            X_b, y_b = X_train[idx], y_train[idx]
            s_b = X_b.dot(w_mb32) + b_mb32
            probs = sigmoid(-y_b*s_b)
            grad_w = (-1/batch32)*(X_b.T @ (y_b*probs)) + lam*w_mb32
            grad_b = (-1/batch32)*np.sum(y_b*probs)
            w_mb32 -= eta*grad_w
            b_mb32 -= eta*grad_b

        # ----- Mini-batch 64 -----
        for t in range(n_iter_cv):
            idx = np.random.choice(n_train, batch64, replace=False)
            X_b, y_b = X_train[idx], y_train[idx]
            s_b = X_b.dot(w_mb64) + b_mb64
            probs = sigmoid(-y_b*s_b)
            grad_w = (-1/batch64)*(X_b.T @ (y_b*probs)) + lam*w_mb64
            grad_b = (-1/batch64)*np.sum(y_b*probs)
            w_mb64 -= eta*grad_w
            b_mb64 -= eta*grad_b

        # ----- SAGA -----
        for t in range(n_iter_cv):
            i_t = np.random.randint(0, n_train)
            x_i, y_i = X_train[i_t], y_train[i_t]
            s_i = np.dot(w_saga, x_i) + b_saga
            g_i_t = -y_i*sigmoid(-y_i*s_i)*x_i + lam*w_saga
            g_bar = np.mean(g_old, axis=0)
            w_saga -= eta*(g_i_t - g_old[i_t] + g_bar)
            s_all = X_train.dot(w_saga) + b_saga
            b_grad = -y_i*sigmoid(-y_i*s_i) + np.mean(-y_train*sigmoid(-y_train*s_all))
            b_saga -= eta*b_grad
            g_old[i_t] = g_i_t

        # ----- Evaluate metrics per fold -----
        algorithms = {
            "GD": (w_gd, b_gd),
            "SGD": (w_sgd, b_sgd),
            "Mini-batch 32": (w_mb32, b_mb32),
            "Mini-batch 64": (w_mb64, b_mb64),
            "SAGA": (w_saga, b_saga)
        }

        for name, (w_alg, b_alg) in algorithms.items():
            y_pred = predict(X_test, w_alg, b_alg)
            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            results_lambda[lam][name].append([acc, prec, rec, f1])


In [30]:
summary_lambda = {}

for lam in lambdas:
    summary = []
    for name in results_lambda[lam]:
        arr = np.array(results_lambda[lam][name])
        summary.append({
            "Algorithm": name,
            "Accuracy": arr[:,0].mean(),
            "Precision": arr[:,1].mean(),
            "Recall": arr[:,2].mean(),
            "F1-score": arr[:,3].mean()
        })
    summary_lambda[lam] = pd.DataFrame(summary).set_index("Algorithm").style.format("{:.4f}")

print("Cross-validation results for different lambda values:")
for lam in lambdas:
    print(f"\nLambda = {lam}")
    display(summary_lambda[lam])
    

Cross-validation results for different lambda values:

Lambda = 0.001


,Accuracy,Precision,Recall,F1-score
Algorithm,,,,
GD,0.9771,0.9723,0.9918,0.9818
SGD,0.9771,0.9723,0.9918,0.9819
Mini-batch 32,0.9771,0.9723,0.9918,0.9818
Mini-batch 64,0.9754,0.9723,0.9890,0.9804
SAGA,0.9754,0.9644,0.9974,0.9804



Lambda = 0.01


,Accuracy,Precision,Recall,F1-score
Algorithm,,,,
GD,0.9771,0.9723,0.9918,0.9818
SGD,0.9701,0.9747,0.9780,0.9761
Mini-batch 32,0.9718,0.9694,0.9860,0.9775
Mini-batch 64,0.9754,0.9696,0.9918,0.9804
SAGA,0.9701,0.9641,0.9888,0.9762



Lambda = 0.1


,Accuracy,Precision,Recall,F1-score
Algorithm,,,,
GD,0.9754,0.9644,0.9974,0.9804
SGD,0.9630,0.9562,0.9864,0.9708
Mini-batch 32,0.9754,0.9644,0.9974,0.9804
Mini-batch 64,0.9736,0.9618,0.9974,0.9792
SAGA,0.9666,0.9541,0.9946,0.9737



Lambda = 1.0


,Accuracy,Precision,Recall,F1-score
Algorithm,,,,
GD,0.9349,0.9123,0.9918,0.9502
SGD,0.9437,0.9219,0.9946,0.9567
Mini-batch 32,0.9384,0.9126,0.9972,0.9529
Mini-batch 64,0.9349,0.9123,0.9918,0.9502
SAGA,0.9314,0.9054,0.9944,0.9476


# <span style="color:blue">Results – Breast Cancer Wisconsin Dataset</span>

## <span style="color:green">1. Model Overview</span>

- **Goal:** Predict `target` (Malignant / Benign) using logistic regression  
- **Features Used:** All 30 numeric features (standardized)  
- **Model Type:** Logistic Regression + L2 penalty (λ = 0.1)  
- **Training:** GD, SGD, Mini-Batch GD (32, 64), SAGA  

---

## <span style="color:green">2. Model Performance – k-Fold CV (5-fold)</span>

| Algorithm        | Accuracy | Precision | Recall | F1-score |
|-----------------|----------|-----------|--------|----------|
| <span style="color:blue">GD</span>              | 0.9469   | 0.9178    | 1.0000 | 0.9571   |
| <span style="color:orange">SGD</span>           | 0.9646   | 0.9565    | 0.9851 | 0.9706   |
| <span style="color:green">Mini-batch 32</span>  | 0.9558   | 0.9306    | 1.0000 | 0.9640   |
| <span style="color:purple">Mini-batch 64</span>  | 0.9558   | 0.9306    | 1.0000 | 0.9640   |
| <span style="color:red">SAGA</span>             | 0.9558   | 0.9306    | 1.0000 | 0.9640   |

> **Observation:** SGD slightly better accuracy; all other algorithms stable. Recall = 1 for most models → all malignant cases detected.

---

## <span style="color:green">3. Train & Test Loss (Final)</span>

| Algorithm        | Train Loss | Test Loss |
|-----------------|------------|-----------|
| <span style="color:blue">GD</span>              | 0.1999     | 0.1858    |
| <span style="color:orange">SGD</span>           | 0.2009     | 0.1877    |
| <span style="color:green">Mini-batch 32</span>  | 0.1999     | 0.1861    |
| <span style="color:purple">Mini-batch 64</span>  | 0.1999     | 0.1855    |
| <span style="color:red">SAGA</span>             | 0.1999     | 0.1857    |

> **Observation:** Very small difference between train and test loss → minimal overfitting.  

---

## <span style="color:green">4. λ (Regularization) Effect on Accuracy – k-Fold CV</span>

| λ     | GD    | SGD    | Mini-batch 32 | Mini-batch 64 | SAGA   |
|-------|-------|--------|---------------|---------------|--------|
| 0.001 | 0.948 | 0.965  | 0.957         | 0.956         | 0.956  |
| 0.01  | 0.946 | 0.964  | 0.956         | 0.956         | 0.956  |
| 0.1   | 0.946 | 0.964  | 0.955         | 0.955         | 0.955  |
| 1.0   | 0.942 | 0.960  | 0.952         | 0.952         | 0.952  |

> **Observation:** Higher λ → slightly lower accuracy (weights shrink more) but improves stability; lower λ → slightly better fit.  

---

## <span style="color:green">5. Key Insights</span>

1. Logistic regression with L2 penalty performs extremely well on this dataset due to **high feature separability**.  
2. **SGD converges slightly faster** and achieves highest average accuracy.  
3. Mini-batch variants (32, 64) and SAGA are **stable** across folds.  
4. λ controls weight magnitude: small λ → more flexible fit; large λ → more regularization, slightly lower accuracy.  
5. **Overfitting minimal**, even without additional transformations.
